AUTHOR : Sofia Boumahrat

DATE : 22/11/2022

PROJECT REMOOR SENSING

# Imports

In [2]:
import os
import numpy as np
from glob import glob

import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd
from sklearn.model_selection import train_test_split

from data_utils import index_files, build_dataset
from eval_utils import plot_confusion_matrix, plot_to_image
from train_utils import SparsePrecision, SparseRecall, BackupAndRestore

# Set the parameters

In [4]:
image_size = (250, 250)
batch_size = 32
SEED = 1137

# Load the data

In [5]:
data_path="ImagesLandesOccitanie"
img_dir = os.path.join(data_path)

    # Images saved in TIFF format
img_files = glob(os.path.join(img_dir, "**/*.tif"), recursive=True)

img_labels = [path.split("/")[-2] for path in img_files]
class_names = np.unique(img_labels)
class_ids = dict([(l, ix) for ix, l in enumerate(class_names)])



files_train, files_test, labels_train, labels_test = train_test_split(
    img_files, img_labels,
    stratify=img_labels,
    test_size=0.50,
    random_state=SEED
)

# Functions
### Get images from a directory based on a classement and put them in a new one

In [6]:
import shutil

def getfilesnames(df,th,files_test):
    ''' Get the name of files where output probabilities is inferior to a rate
    Input:
        th: threshold between ]0,1[
        df: dataframe containing the output probabilities
        files_test: files from the test set
    Output:
        file_names : an array of filenames where their output probability is inferior to th
    '''
    mask_threshold=df.max_prob<th
    index_mask=df[mask_threshold].index.values
    file_names = [files_test[i] for i in index_mask]
    return (file_names)

def movefiles(files,destination):
    '''  Move files from a directory to another
    Input:
        files: files to move
        destination: destination directory for the files
            example :"/Users/boumahrat/Desktop/EPF/5A/remoor_sensing/test_model/Images_to_label_hand"
    '''
    for name in files:
        shutil.copy(name, destination)


# Example of application

In [7]:
#EXAMPLE OF USE

#reproduce the datasets with the resulting predictions
A=['culture_annuelle', 'culture_perenne','pelouse','lande_fermee','lande_fermee']
B=[0.48,0.92,0.20,0.78,0.54]
data = {'prediction': A, 'max_prob': B}
df=pd.DataFrame(data)

result=getfilesnames(df,0.55,files_test)
destination="/Users/boumahrat/Desktop/EPF/5A/remoor_sensing/test_model/Images_to_label_hand"
movefiles(result,destination)